<center>
<b><i><font color="Sky Blue" size="8">T20 Cricket Analysis</font></i></b>
</center>

<h2><b><font color="gold">Introduction</font></b></h2>

In [196]:
# Mounting Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [198]:
# Importing Requires Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [200]:
# Path for files from drive
T20_ball_file = '/content/drive/MyDrive/Cricket_Project/data/model_ready/Ball By Ball/t20_ball_by_ball.csv'
T20_info_file = '/content/drive/MyDrive/Cricket_Project/data/model_ready/Info/t20_info.csv'
T20_summary_file = '/content/drive/MyDrive/Cricket_Project/data/model_ready/Summary/t20_summary.csv'

# Loading the CSV's into a DataFrame
df_T20_ball = pd.read_csv(T20_ball_file, low_memory=False)
df_T20_info = pd.read_csv(T20_info_file, low_memory=False)
df_T20_summary = pd.read_csv(T20_summary_file, low_memory=False)

In [202]:
# Verifying the shape of Dataframes

print(f"Ball By Ball shape: {df_T20_ball.shape}")
print(f"Info shape: {df_T20_info.shape}")
print(f"Summary shape: {df_T20_summary.shape}")

Ball By Ball shape: (617127, 26)
Info shape: (2730, 4769)
Summary shape: (2730, 21)


<h2><b><font color="gold">Data Cleaning</font></b></h2>

In [ ]:
# Checking for missing values in each column
missing_counts = df_T20_ball.isnull().sum()
print(missing_counts)



In [ ]:
# Remove columns with all missing values

t20_cols_with_all_missing = [
    col for col in df_T20_ball.columns if df_T20_ball[col].isnull().all()
]
df_T20_ball.drop(columns=t20_cols_with_all_missing, inplace=True)

# Fill missing values in main numeric columns with 0
df_T20_ball['runs.batter'] = df_T20_ball['runs.batter'].fillna(0)
df_T20_ball['runs.extras'] = df_T20_ball['runs.extras'].fillna(0)
df_T20_ball['runs.total'] = df_T20_ball['runs.total'].fillna(0)

# Fill missing values in extras sub-columns with 0
for col in ['extras.wides', 'extras.noballs', 'extras.legbyes', 'extras.byes', 'extras.penalty']:
    if col in df_T20_ball.columns:
        df_T20_ball[col] = df_T20_ball[col].fillna(0)

# Create is_wicket column: 1 if a wicket fell on this ball, else 0
df_T20_ball['is_wicket'] = df_T20_ball['wicket.kind'].notnull().astype(int)



In [ ]:
# Renaming columns for better readability

df_T20_ball = df_T20_ball.rename(columns={
    'runs.batter': 'batsman_runs',
    'runs.extras': 'extras',
    'runs.total': 'total_runs'
})


<h2><b><font color="gold">Data Pre-Processing</font></b></h2>

In [ ]:
# Standardizing the Match ID Column Name

for df in [df_T20_ball, df_T20_info]:
    if 'Match Id' in df.columns:
        df.rename(columns={'Match Id': 'Match_ID'}, inplace=True)

In [ ]:
# Removing unnesasary columns before merging

required_info_cols = [
    'Match_ID', 'balls_per_over', 'city', 'dates', 'event.name',
    'event.match_number', 'gender', 'match_type', 'match_type_number',
    'season', 'teams', 'player_of_match', 'outcome.winner', 'outcome.by.runs',
    'outcome.by.wickets', 'outcome.result', 'outcome.summary', 'venue'
]

# Use only columns present in your file

df_T20_info = df_T20_info[[col for col in required_info_cols if col in df_T20_info.columns]].copy()

In [ ]:
# Merging Ball data and Info data

df_T20_ball = df_T20_ball.merge(df_T20_info, on='Match_ID', how='left')

<h2><b><font color="gold">Finding batting team and opponent team for better analysis</font></b></h2>

In [ ]:
# Finding opponent team

def get_opposition(row):

    teams = row['teams']
    if isinstance(teams, list):
        pass
    elif isinstance(teams, str):
        teams = [t.strip() for t in teams.split(',')]
    else:
        return None

    if row['batting_team'] == teams[0]:
        return teams[1]
    elif row['batting_team'] == teams[1]:
        return teams[0]
    else:
        return None

df_T20_ball['opposition'] = df_T20_ball.apply(get_opposition, axis=1)


In [ ]:
# Finding Batting Team

def find_batting_team(row):
    """
    This function returns the team name (from 'team_list') that the 'batter' plays for,
    using the players.<team_name> columns in the same DataFrame.
    """
    for t in row['team_list']:
        player_col = f"players.{t}"
        players = row.get(player_col, [])
        if isinstance(players, str):
            import ast
            try:
                players = ast.literal_eval(players)
            except Exception:
                players = [p.strip() for p in players.split(',')]
        if row['batter'] in players:
            return t
    return None

<h2><b><font color="gold">Overall analysis visualizations</font></b></h2>


In [ ]:
# Most Runs scored Battters in T20

plt.figure(figsize=(12,5))
top_batsmen = df_T20_ball.groupby('batter')['batsman_runs'].sum().sort_values(ascending=False).head(10)
sns.barplot(
    x=top_batsmen.values,
    y=top_batsmen.index,
    hue=top_batsmen.index,
    palette="viridis",
    legend=False
)

plt.title("Top 10 Most T20 Run Scorers ", color = 'red')
plt.xlabel("Total Runs",  color='blue', fontsize=12)
plt.ylabel("Batter", color='green', fontsize=12)
plt.show()


In [ ]:
# Most wicket taking bowlers in T20

plt.figure(figsize=(12,5))
top_bowlers = df_T20_ball[df_T20_ball['is_wicket']==1].groupby('bowler').size().sort_values(ascending=False).head(10)
sns.barplot(
    x=top_bowlers.values,
    y=top_bowlers.index,
    hue=top_bowlers.index,
    palette="viridis",
    legend=False
)
plt.xlabel('Wickets', color='blue', fontsize=12)
plt.ylabel('Bowler', color='green', fontsize=12)
plt.title('Top 10 T20 Bowlers by Wickets', color = 'red')
plt.show()

In [ ]:
# Total runs scored team for match

runs_per_innings = df_T20_ball.groupby(['Match_ID', 'batting_team'])['total_runs'].sum().reset_index()

plt.figure(figsize=(16,6))
sns.boxplot(x='batting_team', y='total_runs', data=runs_per_innings)
plt.title("Distribution of Total Runs per Match by Batting Team in T20's", color = 'red')
plt.xlabel("Batting Team",  color='blue', fontsize=12)
plt.ylabel("Runs", color='green', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Total Wickets taken by Bowling team per match

wickets_per_innings = df_T20_ball.groupby(['Match_ID', 'team'])['is_wicket'].sum().reset_index()

plt.figure(figsize=(16,6))
sns.boxplot(x='team', y='is_wicket', data=wickets_per_innings)
plt.title("Distribution of Total Wickets Taken By Bowling Team Per Match in T20's", color = 'red')
plt.xlabel("Bowling Team", color='blue', fontsize=12)
plt.ylabel("Wickets", color='green', fontsize=12)
plt.xticks(rotation=90)
plt.show()


In [ ]:
# Number of matches won by each team

df_T20_info['outcome.winner'] = df_T20_info['outcome.winner'].fillna('No Result')
plt.figure(figsize=(16,6))
winner_counts = df_T20_info['outcome.winner'].value_counts().reset_index()
winner_counts.columns = ['Winner', 'Count']
ax = sns.barplot(data=winner_counts, x='Winner', y='Count')
plt.title("Number of Matches Won by Each Team", color = 'red')
plt.xlabel("Winning Team", color='blue', fontsize=12)
plt.ylabel("Number of Matches", color='green', fontsize=12)
plt.xticks(rotation=75)
plt.tight_layout()
plt.show()

<h2><b><font color="gold">venue Mapping for further Analysis</font></b></h2>

In [ ]:
unique_venues = sorted(df_T20_ball['venue'].unique())
for v in unique_venues:
    print(v)


In [ ]:
venue_mapping_T20 = {
    # Achimota Senior Secondary School
    "Achimota Senior Secondary School A Field, Accra": "Achimota Senior Secondary School Field, Accra",
    "Achimota Senior Secondary School B Field, Accra": "Achimota Senior Secondary School Field, Accra",
    "Achimota Senior Secondary School Field, Accra": "Achimota Senior Secondary School Field, Accra",

    # Al Amerat Cricket Ground Oman Cricket
    "Al Amerat Cricket Ground Oman Cricket (Ministry Turf 1)": "Al Amerat Cricket Ground Oman Cricket",
    "Al Amerat Cricket Ground Oman Cricket (Ministry Turf 2)": "Al Amerat Cricket Ground Oman Cricket",
    "Al Amerat Cricket Ground Oman Cricket": "Al Amerat Cricket Ground Oman Cricket",

    # Amini Park
    "Amini Park": "Amini Park",
    "Amini Park, Port Moresby": "Amini Park",

    # Arnos Vale Ground
    "Arnos Vale Ground, Kingstown": "Arnos Vale Ground",
    "Arnos Vale Ground, Kingstown, St Vincent": "Arnos Vale Ground",
    "Arnos Vale Ground": "Arnos Vale Ground",

    # Arun Jaitley Stadium / Feroz Shah Kotla
    "Arun Jaitley Stadium, Delhi": "Arun Jaitley Stadium",
    "Arun Jaitley Stadium": "Arun Jaitley Stadium",
    "Feroz Shah Kotla": "Arun Jaitley Stadium",

    # Barabati Stadium
    "Barabati Stadium": "Barabati Stadium",
    "Barabati Stadium, Cuttack": "Barabati Stadium",

    # Barsapara Cricket Stadium
    "Barsapara Cricket Stadium": "Barsapara Cricket Stadium",
    "Barsapara Cricket Stadium, Guwahati": "Barsapara Cricket Stadium",

    # Bay Oval
    "Bay Oval": "Bay Oval",
    "Bay Oval, Mount Maunganui": "Bay Oval",

    # Bellerive Oval
    "Bellerive Oval": "Bellerive Oval",
    "Bellerive Oval, Hobart": "Bellerive Oval",

    # Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium
    "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow": "Ekana Cricket Stadium",
    "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium": "Ekana Cricket Stadium",
    "Ekana Cricket Stadium": "Ekana Cricket Stadium",

    # Bready
    "Bready": "Bready Cricket Club Ground",
    "Bready Cricket Club, Magheramason": "Bready Cricket Club Ground",
    "Bready Cricket Club, Magheramason, Bready": "Bready Cricket Club Ground",
    "Bready Cricket Club Ground": "Bready Cricket Club Ground",

    # Brisbane Cricket Ground
    "Brisbane Cricket Ground, Woolloongabba": "Brisbane Cricket Ground",
    "Brisbane Cricket Ground, Woolloongabba, Brisbane": "Brisbane Cricket Ground",
    "Brisbane Cricket Ground": "Brisbane Cricket Ground",

    # Central Broward Regional Park Stadium Turf Ground
    "Central Broward Regional Park Stadium Turf Ground": "Central Broward Regional Park Stadium",
    "Central Broward Regional Park Stadium Turf Ground, Lauderhill": "Central Broward Regional Park Stadium",
    "Central Broward Regional Park Stadium": "Central Broward Regional Park Stadium",

    # Civil Service Cricket Club, Stormont
    "Civil Service Cricket Club, Stormont": "Stormont",
    "Civil Service Cricket Club, Stormont, Belfast": "Stormont",
    "Stormont": "Stormont",

    # Daren Sammy National Cricket Stadium
    "Daren Sammy National Cricket Stadium, Gros Islet, St Lucia": "Daren Sammy National Cricket Stadium",
    "Darren Sammy National Cricket Stadium, St Lucia": "Daren Sammy National Cricket Stadium",
    "Daren Sammy National Cricket Stadium": "Daren Sammy National Cricket Stadium",

    # Desert Springs Cricket Ground
    "Desert Springs Cricket Ground": "Desert Springs Cricket Ground",
    "Desert Springs Cricket Ground, Almeria": "Desert Springs Cricket Ground",

    # Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium
    "Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam": "ACA-VDCA Cricket Stadium",
    "Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium": "ACA-VDCA Cricket Stadium",
    "ACA-VDCA Cricket Stadium": "ACA-VDCA Cricket Stadium",

    # Eden Gardens
    "Eden Gardens": "Eden Gardens",
    "Eden Gardens, Kolkata": "Eden Gardens",

    # Edgbaston
    "Edgbaston": "Edgbaston",
    "Edgbaston, Birmingham": "Edgbaston",

    # Gaddafi Stadium
    "Gaddafi Stadium": "Gaddafi Stadium",
    "Gaddafi Stadium, Lahore": "Gaddafi Stadium",

    # Gahanga International Cricket Stadium
    "Gahanga International Cricket Stadium, Rwanda": "Gahanga International Cricket Stadium",
    "Gahanga International Cricket Stadium. Rwanda": "Gahanga International Cricket Stadium",
    "Gahanga International Cricket Stadium": "Gahanga International Cricket Stadium",

    # Grange Cricket Club Ground
    "Grange Cricket Club, Raeburn Place": "Grange Cricket Club Ground",
    "Grange Cricket Club Ground, Raeburn Place": "Grange Cricket Club Ground",
    "Grange Cricket Club Ground, Raeburn Place, Edinburgh": "Grange Cricket Club Ground",
    "Grange Cricket Club Ground": "Grange Cricket Club Ground",

    # Greenfield International Stadium
    "Greenfield International Stadium, Thiruvananthapuram": "Greenfield International Stadium",
    "Greenfield International Stadium": "Greenfield International Stadium",

    # Hagley Oval
    "Hagley Oval": "Hagley Oval",
    "Hagley Oval, Christchurch": "Hagley Oval",

    # Holkar Cricket Stadium
    "Holkar Cricket Stadium": "Holkar Cricket Stadium",
    "Holkar Cricket Stadium, Indore": "Holkar Cricket Stadium",

    # JSCA International Stadium Complex
    "JSCA International Stadium Complex, Ranchi": "JSCA International Stadium Complex",
    "JSCA International Stadium Complex": "JSCA International Stadium Complex",

    # Kensington Oval
    "Kensington Oval": "Kensington Oval",
    "Kensington Oval, Bridgetown": "Kensington Oval",
    "Kensington Oval, Bridgetown, Barbados": "Kensington Oval",

    # King George V Sports Ground
    "King George V Sports Ground": "King George V Sports Ground",
    "King George V Sports Ground, Castel": "King George V Sports Ground",

    # Kingsmead
    "Kingsmead": "Kingsmead",
    "Kingsmead, Durban": "Kingsmead",

    # M Chinnaswamy Stadium
    "M Chinnaswamy Stadium": "M Chinnaswamy Stadium",
    "M Chinnaswamy Stadium, Bangalore": "M Chinnaswamy Stadium",
    "M Chinnaswamy Stadium, Bengaluru": "M Chinnaswamy Stadium",
    "M.Chinnaswamy Stadium": "M Chinnaswamy Stadium",

    # MA Chidambaram Stadium
    "MA Chidambaram Stadium, Chepauk": "MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chepauk, Chennai": "MA Chidambaram Stadium",
    "MA Chidambaram Stadium": "MA Chidambaram Stadium",

    # Maharashtra Cricket Association Stadium
    "Maharashtra Cricket Association Stadium": "Maharashtra Cricket Association Stadium",
    "Maharashtra Cricket Association Stadium, Pune": "Maharashtra Cricket Association Stadium",

    # Moara Vlasiei Cricket Ground
    "Moara Vlasiei Cricket Ground, Ilfov County": "Moara Vlasiei Cricket Ground",
    "Moara Vlasiei Cricket Ground": "Moara Vlasiei Cricket Ground",
    "Moara Vlasiei Cricket Ground, Ilfov County, Ilfov": "Moara Vlasiei Cricket Ground",

    # New Wanderers Stadium / Wanderers
    "New Wanderers Stadium": "Wanderers",
    "Wanderers": "Wanderers",
    "Wanderers Cricket Ground": "Wanderers",
    "Wanderers Cricket Ground, Windhoek": "Wanderers",
    "The Wanderers Stadium": "Wanderers",
    "The Wanderers Stadium, Johannesburg": "Wanderers",

    # Narendra Modi Stadium
    "Narendra Modi Stadium": "Narendra Modi Stadium",
    "Narendra Modi Stadium, Ahmedabad": "Narendra Modi Stadium",

    # Old Trafford
    "Old Trafford": "Old Trafford",
    "Old Trafford, Manchester": "Old Trafford",

    # R Premadasa Stadium
    "R Premadasa Stadium": "R Premadasa Stadium",
    "R Premadasa Stadium, Colombo": "R Premadasa Stadium",
    "R.Premadasa Stadium, Khettarama": "R Premadasa Stadium",

    # Rajiv Gandhi International Stadium
    "Rajiv Gandhi International Stadium, Uppal": "Rajiv Gandhi International Stadium",
    "Rajiv Gandhi International Stadium, Uppal, Hyderabad": "Rajiv Gandhi International Stadium",
    "Rajiv Gandhi International Stadium": "Rajiv Gandhi International Stadium",

    # Saurashtra Cricket Association Stadium
    "Saurashtra Cricket Association Stadium": "Saurashtra Cricket Association Stadium",
    "Saurashtra Cricket Association Stadium, Rajkot": "Saurashtra Cricket Association Stadium",

    # Sharjah Cricket Stadium
    "Sharjah Cricket Stadium": "Sharjah Cricket Stadium",

    # Sheikh Zayed Stadium / Zayed Cricket Stadium
    "Zayed Cricket Stadium, Abu Dhabi": "Sheikh Zayed Stadium",
    "Sheikh Zayed Stadium": "Sheikh Zayed Stadium",

    # SuperSport Park
    "SuperSport Park": "SuperSport Park",
    "SuperSport Park, Centurion": "SuperSport Park",

    # Sydney Cricket Ground
    "Sydney Cricket Ground": "Sydney Cricket Ground",

    # Trent Bridge
    "Trent Bridge": "Trent Bridge",
    "Trent Bridge, Nottingham": "Trent Bridge",

    # Wankhede Stadium
    "Wankhede Stadium": "Wankhede Stadium",
    "Wankhede Stadium, Mumbai": "Wankhede Stadium",

    # Warner Park
    "Warner Park": "Warner Park",
    "Warner Park, Basseterre": "Warner Park",
    "Warner Park, Basseterre, St Kitts": "Warner Park",
    "Warner Park, St Kitts": "Warner Park",

    # Zahur Ahmed Chowdhury Stadium
    "Zahur Ahmed Chowdhury Stadium": "Zahur Ahmed Chowdhury Stadium",
    "Zahur Ahmed Chowdhury Stadium, Chattogram": "Zahur Ahmed Chowdhury Stadium",

    # All solo/unique venues mapped to themselves:
    "AMI Stadium": "AMI Stadium",
    "Adelaide Oval": "Adelaide Oval",
    "Ballpark Ground, Graz": "Ballpark Ground, Graz",
    "Beausejour Stadium, Gros Islet": "Beausejour Stadium, Gros Islet",
    "Belgrano Athletic Club Ground, Buenos Aires": "Belgrano Athletic Club Ground, Buenos Aires",
    "Bermuda National Stadium": "Bermuda National Stadium",
    "Boland Park": "Boland Park",
    "Brabourne Stadium": "Brabourne Stadium",
    "Buffalo Park": "Buffalo Park",
    "Bulawayo Athletic Club": "Bulawayo Athletic Club",
    "Carrara Oval": "Carrara Oval",
    "Castle Avenue, Dublin": "Castle Avenue, Dublin",
    "Clayton Panama, Panama City": "Clayton Panama, Panama City",
    "College Field": "College Field",
    "College Field, St Peter Port": "College Field, St Peter Port",
    "Coolidge Cricket Ground, Antigua": "Coolidge Cricket Ground, Antigua",
    "County Ground": "County Ground",
    "County Ground, Bristol": "County Ground, Bristol",
    "De Beers Diamond Oval": "De Beers Diamond Oval",
    "Dreux Sport Cricket Club": "Dreux Sport Cricket Club",
    "Dubai International Cricket Stadium": "Dubai International Cricket Stadium",
    "Eden Park": "Eden Park",
    "Eden Park, Auckland": "Eden Park, Auckland",
    "Entebbe Cricket Oval": "Entebbe Cricket Oval",
    "Europa Sports Complex": "Europa Sports Complex",
    "Faleata Oval No 2, Apia": "Faleata Oval No 2, Apia",
    "GB Oval, Szodliget": "GB Oval, Szodliget",
    "GMHBA Stadium, South Geelong, Victoria": "GMHBA Stadium, South Geelong, Victoria",
    "Gelephu International Cricket Ground": "Gelephu International Cricket Ground",
    "Goldenacre, Edinburgh": "Goldenacre, Edinburgh",
    "Grainville, St Saviour": "Grainville, St Saviour",
    "Grand Prairie Stadium, Dallas": "Grand Prairie Stadium, Dallas",
    "Green Park": "Green Park",
    "Gucherre Cricket Ground": "Gucherre Cricket Ground",
    "Guernsey Rovers Athletic Club Ground, Port Soif": "Guernsey Rovers Athletic Club Ground, Port Soif",
    "Gymkhana Club Ground": "Gymkhana Club Ground",
    "Gymkhana Club Ground, Dar-es-Salaam": "Gymkhana Club Ground, Dar-es-Salaam",
    "Gymkhana Club Ground, Nairobi": "Gymkhana Club Ground, Nairobi",
    "Happy Valley Ground": "Happy Valley Ground",
    "Harare Sports Club": "Harare Sports Club",
    "Hazelaarweg": "Hazelaarweg",
    "Headingley, Leeds": "Headingley, Leeds",
    "Himachal Pradesh Cricket Association Stadium": "Himachal Pradesh Cricket Association Stadium",
    "Himachal Pradesh Cricket Association Stadium, Dharamsala": "Himachal Pradesh Cricket Association Stadium, Dharamsala",
    "Hurlingham Club Ground, Buenos Aires": "Hurlingham Club Ground, Buenos Aires",
    "ICC Academy": "ICC Academy",
    "ICC Academy Ground No 2": "ICC Academy Ground No 2",
    "ICC Academy, Dubai": "ICC Academy, Dubai",
    "ICC Global Cricket Academy": "ICC Global Cricket Academy",
    "Independence Park, Port Vila": "Independence Park, Port Vila",
    "Indian Association Ground": "Indian Association Ground",
    "Indian Association Ground, Singapore": "Indian Association Ground, Singapore",
    "Integrated Polytechnic Regional Centre": "Integrated Polytechnic Regional Centre",
    "Ishoj Cricket Club, Vejledalen": "Ishoj Cricket Club, Vejledalen",
    "Jade Stadium": "Jade Stadium",
    "Jimmy Powell Oval, Cayman Islands": "Jimmy Powell Oval, Cayman Islands",
    "Jinja Cricket Ground": "Jinja Cricket Ground",
    "John Davies Oval, Queenstown": "John Davies Oval, Queenstown",
    "Kennington Oval": "Kennington Oval",
    "Kennington Oval, London": "Kennington Oval, London",
    "Kerava National Cricket Ground": "Kerava National Cricket Ground",
    "Khan Shaheb Osman Ali Stadium": "Khan Shaheb Osman Ali Stadium",
    "Koge Cricket Club": "Koge Cricket Club",
    "Kyambogo Cricket Oval": "Kyambogo Cricket Oval",
    "La Manga Club Bottom Ground": "La Manga Club Bottom Ground",
    "La Manga Club Top Ground": "La Manga Club Top Ground",
    "Lisicji Jarak Cricket Ground": "Lisicji Jarak Cricket Ground",
    "Lord's": "Lord's",
    "Los Reyes Polo Club": "Los Reyes Polo Club",
    "Lugogo Cricket Oval": "Lugogo Cricket Oval",
    "Mahinda Rajapaksa International Cricket Stadium, Sooriyawewa": "Mahinda Rajapaksa International Cricket Stadium, Sooriyawewa",
    "Malahide, Dublin": "Malahide, Dublin",
    "Mangaung Oval": "Mangaung Oval",
    "Manuka Oval": "Manuka Oval",
    "Manuka Oval, Canberra": "Manuka Oval, Canberra",
    "Maple Leaf North-East Ground, King City": "Maple Leaf North-East Ground, King City",
    "Maple Leaf North-West Ground": "Maple Leaf North-West Ground",
    "Maple Leaf North-West Ground, King City": "Maple Leaf North-West Ground, King City",
    "Marsa Sports Club": "Marsa Sports Club",
    "McLean Park": "McLean Park",
    "McLean Park, Napier": "McLean Park, Napier",
    "Meersen, Gent": "Meersen, Gent",
    "Melbourne Cricket Ground": "Melbourne Cricket Ground",
    "Mission Road Ground, Mong Kok": "Mission Road Ground, Mong Kok",
    "Mission Road Ground, Mong Kok, Hong Kong": "Mission Road Ground, Mong Kok, Hong Kong",
    "Mladost Cricket Ground, Zagreb": "Mladost Cricket Ground, Zagreb",
    "Moara Vlasiei Cricket Ground": "Moara Vlasiei Cricket Ground",
    "Moara Vlasiei Cricket Ground, Ilfov County": "Moara Vlasiei Cricket Ground, Ilfov County",
    "Mombasa Sports Club Ground": "Mombasa Sports Club Ground",
    "Moses Mabhida Stadium": "Moses Mabhida Stadium",
    "Mulpani Cricket Ground": "Mulpani Cricket Ground",
    "Namibia Cricket Ground, Windhoek": "Namibia Cricket Ground, Windhoek",
    "Nassau County International Cricket Stadium, New York": "Nassau County International Cricket Stadium, New York",
    "National Cricket Stadium, Grenada": "National Cricket Stadium, Grenada",
    "National Cricket Stadium, St George's, Grenada": "National Cricket Stadium, St George's, Grenada",
    "National Sports Academy, Sofia": "National Sports Academy, Sofia",
    "National Stadium": "National Stadium",
    "National Stadium, Hamilton": "National Stadium, Hamilton",
    "National Stadium, Karachi": "National Stadium, Karachi",
    "Newlands": "Newlands",
    "Nigeria Cricket Federation Oval 1, Abuja": "Nigeria Cricket Federation Oval 1, Abuja",
    "Nigeria Cricket Federation Oval 2, Abuja": "Nigeria Cricket Federation Oval 2, Abuja",
    "OUTsurance Oval": "OUTsurance Oval",
    "P Sara Oval": "P Sara Oval",
    "Pallekele International Cricket Stadium": "Pallekele International Cricket Stadium",
    "Perth Stadium": "Perth Stadium",
    "Pierre Werner Cricket Ground": "Pierre Werner Cricket Ground",
    "Prairie View Cricket Complex": "Prairie View Cricket Complex",
    "Providence Stadium": "Providence Stadium",
    "Providence Stadium, Guyana": "Providence Stadium, Guyana",
    "Punjab Cricket Association IS Bindra Stadium, Mohali": "Punjab Cricket Association IS Bindra Stadium, Mohali",
    "Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh": "Punjab Cricket Association IS Bindra Stadium, Mohali",
    "Punjab Cricket Association Stadium, Mohali": "Punjab Cricket Association Stadium, Mohali",
    "Queen's Park Oval, Port of Spain": "Queen's Park Oval, Port of Spain",
    "Queens Sports Club": "Queens Sports Club",
    "Queens Sports Club, Bulawayo": "Queens Sports Club, Bulawayo",
    "Rangiri Dambulla International Stadium": "Rangiri Dambulla International Stadium",
    "Rawalpindi Cricket Stadium": "Rawalpindi Cricket Stadium",
    "Riverside Ground": "Riverside Ground",
    "Riverside Ground, Chester-le-Street": "Riverside Ground, Chester-le-Street",
    "Roma Cricket Ground": "Roma Cricket Ground",
    "Royal Brussels Cricket Club Ground, Waterloo": "Royal Brussels Cricket Club Ground, Waterloo",
    "Ruaraka Sports Club Ground, Nairobi": "Ruaraka Sports Club Ground, Nairobi",
    "Sabina Park, Kingston": "Sabina Park, Kingston",
    "Sabina Park, Kingston, Jamaica": "Sabina Park, Kingston, Jamaica",
    "Sano International Cricket Ground": "Sano International Cricket Ground",
    "Santarem Cricket Ground": "Santarem Cricket Ground",
    "Sao Fernando Polo and Cricket Club": "Sao Fernando Polo and Cricket Club",
    "Sardar Patel Stadium, Motera": "Sardar Patel Stadium, Motera",
    "Saxton Oval": "Saxton Oval",
    "Saxton Oval, Nelson": "Saxton Oval, Nelson",
    "Scott Page Field, Vinor": "Scott Page Field, Vinor",
    "Seddon Park": "Seddon Park",
    "Seddon Park, Hamilton": "Seddon Park, Hamilton",
    "Selangor Turf Club, Kuala Lumpur": "Selangor Turf Club, Kuala Lumpur",
    "Senwes Park": "Senwes Park",
    "Shaheed Veer Narayan Singh International Stadium, Raipur": "Shaheed Veer Narayan Singh International Stadium, Raipur",
    "Sheikh Abu Naser Stadium": "Sheikh Abu Naser Stadium",
    "Shere Bangla National Stadium": "Shere Bangla National Stadium",
    "Shere Bangla National Stadium, Mirpur": "Shere Bangla National Stadium",
    "Shrimant Madhavrao Scindia Cricket Stadium, Gwalior": "Shrimant Madhavrao Scindia Cricket Stadium, Gwalior",
    "Sikh Union Club Ground, Nairobi": "Sikh Union Club Ground, Nairobi",
    "Simar Cricket Ground, Rome": "Simar Cricket Ground, Rome",
    "Simonds Stadium, South Geelong": "Simonds Stadium, South Geelong",
    "Singapore National Cricket Ground": "Singapore National Cricket Ground",
    "Sir Vivian Richards Stadium, North Sound": "Sir Vivian Richards Stadium, North Sound",
    "Sir Vivian Richards Stadium, North Sound, Antigua": "Sir Vivian Richards Stadium, North Sound",
    "Sky Stadium": "Sky Stadium",
    "Sky Stadium, Wellington": "Sky Stadium, Wellington",
    "Solvangs Park, Glostrup": "Solvangs Park, Glostrup",
    "Sophia Gardens": "Sophia Gardens",
    "Sophia Gardens, Cardiff": "Sophia Gardens",
    "Sportpark Het Schootsveld": "Sportpark Het Schootsveld",
    "Sportpark Het Schootsveld, Deventer": "Sportpark Het Schootsveld",
    "Sportpark Maarschalkerweerd": "Sportpark Maarschalkerweerd",
    "Sportpark Maarschalkerweerd, Utrecht": "Sportpark Maarschalkerweerd",
    "Sportpark Westvliet": "Sportpark Westvliet",
    "Sportpark Westvliet, The Hague": "Sportpark Westvliet",
    "St Albans Club, Buenos Aires": "St Albans Club, Buenos Aires",
    "St George's College Ground, Buenos Aires": "St George's College Ground, Buenos Aires",
    "St George's Park": "St George's Park",
    "St George's Park, Gqeberha": "St George's Park, Gqeberha",
    "St Georges Quilmes": "St Georges Quilmes",
    "Stadium Australia": "Stadium Australia",
    "Stars Arena Hofstade, Zemst": "Stars Arena Hofstade, Zemst",
    "Subrata Roy Sahara Stadium": "Subrata Roy Sahara Stadium",
    "Svanholm Park, Brondby": "Svanholm Park, Brondby",
    "Sylhet International Cricket Stadium": "Sylhet International Cricket Stadium",
    "Sylhet Stadium": "Sylhet Stadium",
    "TCA Oval, Blantyre": "TCA Oval, Blantyre",
    "Tafawa Balewa Square Cricket Oval, Lagos": "Tafawa Balewa Square Cricket Oval, Lagos",
    "Terdthai Cricket Ground": "Terdthai Cricket Ground",
    "Terdthai Cricket Ground, Bangkok": "Terdthai Cricket Ground",
    "The Rose Bowl": "The Rose Bowl",
    "The Rose Bowl, Southampton": "The Rose Bowl",
    "The Village, Malahide": "The Village, Malahide",
    "The Village, Malahide, Dublin": "The Village, Malahide",
    "Tikkurila Cricket Ground": "Tikkurila Cricket Ground",
    "Titwood, Glasgow": "Titwood, Glasgow",
    "Tolerance Oval": "Tolerance Oval",
    "Tony Ireland Stadium": "Tony Ireland Stadium",
    "Tribhuvan University International Cricket Ground": "Tribhuvan University International Cricket Ground",
    "Tribhuvan University International Cricket Ground, Kirtipur": "Tribhuvan University International Cricket Ground",
    "UKM-YSD Cricket Oval, Bangi": "UKM-YSD Cricket Oval, Bangi",
    "Udayana Cricket Ground": "Udayana Cricket Ground",
    "United Cricket Club Ground": "United Cricket Club Ground",
    "United Cricket Club Ground, Windhoek": "United Cricket Club Ground, Windhoek",
    "University Oval": "University Oval",
    "University Oval, Dunedin": "University Oval, Dunedin",
    "University of Dar-es-Salaam Ground": "University of Dar-es-Salaam Ground",
    "University of Doha for Science and Technology": "University of Doha for Science and Technology",
    "University of Lagos Cricket Oval": "University of Lagos Cricket Oval",
    "VRA Ground": "VRA Ground",
    "Velden Cricket Ground, Latschach": "Velden Cricket Ground, Latschach",
    "Vidarbha Cricket Association Stadium, Jamtha": "Vidarbha Cricket Association Stadium, Jamtha",
    "Vidarbha Cricket Association Stadium, Jamtha, Nagpur": "Vidarbha Cricket Association Stadium, Jamtha",
    "Warner Park": "Warner Park",
    "West End Park International Cricket Stadium, Doha": "West End Park International Cricket Stadium, Doha",
    "Western Australia Cricket Association Ground": "Western Australia Cricket Association Ground",
    "Westpac Stadium": "Westpac Stadium",
    "White Hill Field, Sandys Parish": "White Hill Field, Sandys Parish",
    "Willowmoore Park, Benoni": "Willowmoore Park, Benoni",
    "Windsor Park, Roseau": "Windsor Park, Roseau",
    "Windsor Park, Roseau, Dominica": "Windsor Park, Roseau",
    "YSD-UKM Cricket Oval, Bangi": "YSD-UKM Cricket Oval, Bangi",
    "Yeonhui Cricket Ground, Incheon": "Yeonhui Cricket Ground, Incheon",
    "Zhejiang University of Technology Cricket Field": "Zhejiang University of Technology Cricket Field"
}


In [ ]:
df_T20_ball['venue'] = df_T20_ball['venue'].map(lambda x: venue_mapping_T20.get(x, x))


<h2><b><font color="gold">Batting Metrics of a Batter</font></b></h2>

In [ ]:
# Basic batting metrics by batter

T20_batting_stats = (
    df_T20_ball.groupby('batter').agg(
        runs_scored = ('batsman_runs', 'sum'),
        balls_faced = ('batsman_runs', 'count'),
        fours = ('batsman_runs', lambda x: (x == 4).sum()),
        sixes = ('batsman_runs', lambda x: (x == 6).sum()),
        dismissals = ('is_wicket', 'sum')
    ).reset_index()
)

In [ ]:
T20_innings_scores = df_T20_ball.groupby(['batter', 'Match_ID'])['batsman_runs'].sum().reset_index()
advanced = (
    T20_innings_scores.groupby('batter').agg(
        high_score = ('batsman_runs', 'max'),
        fifties = ('batsman_runs', lambda x: ((x >= 50) & (x < 100)).sum()),
        hundreds = ('batsman_runs', lambda x: ((x >= 100) & (x < 200)).sum()),
        double_hundreds = ('batsman_runs', lambda x: (x >= 200).sum()),
        score_std = ('batsman_runs', 'std')
    ).reset_index()
)

# Round off score_std with 2 decimal values
advanced['score_std'] = advanced['score_std'].round(2)


In [ ]:
# Merging Basic Metrics and Adavance Metrics

T20_batting_stats = T20_batting_stats.merge(advanced, on='batter', how='left')


In [ ]:
# Post-processing adding strike rate, batting average to the dataframe

T20_batting_stats['strike_rate'] = (T20_batting_stats['runs_scored'] / T20_batting_stats['balls_faced'] * 100).replace([np.inf, np.nan], 0).round(2)
T20_batting_stats['batting_average'] = (T20_batting_stats['runs_scored'] / T20_batting_stats['dismissals']).replace([np.inf, np.nan], 0).round(2)

In [ ]:
# Finding Recent Form from last 5 matches
N = 5
T20_innings_scores['recent_form_runs'] = (
    T20_innings_scores.groupby('batter')['batsman_runs']
    .transform(lambda x: x.shift(1).rolling(window=N, min_periods=1).mean())
)
T20_innings_scores['career_avg_runs'] = (
    T20_innings_scores.groupby('batter')['batsman_runs'].transform(lambda x: x.expanding().mean())
)

<h2><b><font color="gold">Metrics of Batter Vs Opponent</font></b></h2>

In [ ]:
# Metrics of Batter vs Opponent

T20_batter_vs_opp = (
    df_T20_ball.groupby(['batter', 'opposition']).agg(
        innings_played=('Match_ID', pd.Series.nunique),
        runs_scored=('batsman_runs', 'sum'),
        balls_faced=('batsman_runs', 'count'),
        dismissals=('is_wicket', 'sum')
    ).reset_index()
)
T20_batter_vs_opp['batting_average'] = (T20_batter_vs_opp['runs_scored'] /T20_batter_vs_opp['dismissals']).replace([np.inf, np.nan], 0).round(2)
T20_batter_vs_opp['strike_rate'] = (T20_batter_vs_opp['runs_scored'] / T20_batter_vs_opp['balls_faced'] * 100).replace([np.inf, np.nan], 0).round(2)


In [ ]:
# Adding Boundary Percentage & Balls per Boundary to the dataframe

T20_batting_stats['boundary_pct'] = (
    (T20_batting_stats['fours'] + T20_batting_stats['sixes']) / T20_batting_stats['balls_faced'] * 100
).replace([np.inf, np.nan], 0).round(2)

T20_batting_stats['balls_per_boundary'] = (
    T20_batting_stats['balls_faced'] / (T20_batting_stats['fours'] + T20_batting_stats['sixes'])
).replace([np.inf, np.nan], 0).round(2)


In [ ]:
# Wide Format Batter Overall Average Metrics and Average Vs Opponent

T20_batter_vs_opp_pivot = (
    T20_batter_vs_opp.pivot(index='batter', columns='opposition', values='batting_average')
    .add_prefix('avg_vs_')
    .reset_index()
)
batting_summary = T20_batting_stats.merge(T20_batter_vs_opp_pivot, on='batter', how='left')


<h2><b><font color="gold">Metrics of Batter Vs venue</font></b></h2>

In [ ]:
# Runs for each batter at each venue

T20_innings_scores = (
    df_T20_ball.groupby(['batter', 'venue', 'Match_ID'])['batsman_runs']
    .sum()
    .reset_index()
)


In [ ]:
# Counting 50s and 100s at each venue for each batter

T20_fifty_hundred_stats = (
    T20_innings_scores.groupby(['batter', 'venue'])
    .agg(
        fifties=('batsman_runs', lambda x: ((x >= 50) & (x < 100)).sum()),
        hundreds=('batsman_runs', lambda x: ((x >= 100) & (x < 200)).sum()),
        double_hundreds=('batsman_runs', lambda x: (x >= 200).sum())
    )
    .reset_index()
)

In [ ]:
# Core Batting Stats by batter and venue

T20_batter_vs_venue = (
    df_T20_ball.groupby(['batter', 'venue']).agg(
        innings_played=('Match_ID', pd.Series.nunique),
        runs_scored=('batsman_runs', 'sum'),
        balls_faced=('batsman_runs', 'count'),
        dismissals=('is_wicket', 'sum'),
        fours=('batsman_runs', lambda x: (x == 4).sum()),
        sixes=('batsman_runs', lambda x: (x == 6).sum())
    ).reset_index()
)


In [ ]:
# Adding 50s/100s/200s to the venue stats df

T20_batter_vs_venue = T20_batter_vs_venue.merge(
    T20_fifty_hundred_stats, on=['batter', 'venue'], how='left'
)

In [ ]:
# Advanced Batting Metrics

# Finding Batting Average for each venue

T20_batter_vs_venue['batting_average'] = (
    T20_batter_vs_venue['runs_scored'] / T20_batter_vs_venue['dismissals']
).replace([np.inf, np.nan], 0).round(2)

# Finding Strike Rate for each venue

T20_batter_vs_venue['strike_rate'] = (
    T20_batter_vs_venue['runs_scored'] / T20_batter_vs_venue['balls_faced'] * 100
).replace([np.inf, np.nan], 0).round(2)

# Filling NaN for fifties/hundreds/double_hundreds ----
T20_batter_vs_venue[['fifties', 'hundreds', 'double_hundreds']] = T20_batter_vs_venue[[
    'fifties', 'hundreds', 'double_hundreds'
]].fillna(0).astype(int)

<h2><b><font color="gold">Bowling Metrics for Bowler</font></b></h2>

In [ ]:
# A maiden = an over with total_runs == 0 for a bowler in a match

T20_overs_summary = (
    df_T20_ball.groupby(['bowler', 'Match_ID', 'batting_team', 'over'])
    .agg(total_runs_in_over=('total_runs', 'sum'))
    .reset_index()
)
T20_overs_summary['is_maiden'] = (T20_overs_summary['total_runs_in_over'] == 0).astype(int)

T20_maidens_by_bowler = (
    T20_overs_summary.groupby('bowler')['is_maiden']
    .sum()
    .reset_index()
    .rename(columns={'is_maiden': 'maidens'})
)

In [ ]:
# Bowling Metrics for bowler

T20_agg_dict = {
    'balls_bowled': ('bowler', 'count'),
    'runs_conceded': ('total_runs', 'sum'),
    'wickets': ('is_wicket', 'sum'),
    'fours_conceded': ('batsman_runs', lambda x: (x == 4).sum()),
    'sixes_conceded': ('batsman_runs', lambda x: (x == 6).sum()),
}

T20_bowling_stats = df_T20_ball.groupby('bowler').agg(**T20_agg_dict).reset_index()
T20_bowling_stats = T20_bowling_stats.merge(T20_maidens_by_bowler, on='bowler', how='left')
T20_bowling_stats['maidens'] = T20_bowling_stats['maidens'].fillna(0).astype(int)

In [ ]:
# Per-Innings Bowling Summary

T20_innings_bowling = (
    df_T20_ball.groupby(['bowler', 'Match_ID', 'batting_team'])
    .agg(
        runs_conceded=('total_runs', 'sum'),
        wickets=('is_wicket', 'sum'),
        balls_bowled=('bowler', 'count')
    ).reset_index()
)

In [ ]:
# Best innings figures (most wickets and then fewest runs)

T20_best_innings = (
    T20_innings_bowling.loc[
        T20_innings_bowling.groupby('bowler')['wickets'].idxmax()
    ][['bowler', 'wickets', 'runs_conceded']]
    .rename(columns={'wickets': 'best_innings_wickets', 'runs_conceded': 'best_innings_runs_conceded'})
)

In [ ]:
# 5WI, std dev
advanced_bowl = (
    T20_innings_bowling.groupby('bowler').agg(
        five_wicket_hauls=('wickets', lambda x: (x >= 5).sum()),
        wickets_std=('wickets', 'std')
    ).reset_index()
)
advanced_bowl['wickets_std'] = advanced_bowl['wickets_std'].replace([np.inf, np.nan], 0).round(2)

In [ ]:
# Merging advanced metrics

advanced_bowl = advanced_bowl.merge(T20_best_innings, on='bowler', how='left')


In [ ]:
# Merging Basic and Advanced metrics

T20_bowling_stats = T20_bowling_stats.merge(advanced_bowl, on='bowler', how='left')
T20_bowling_stats['best_innings_wickets'] = T20_bowling_stats['best_innings_wickets'].fillna(0).astype(int)

In [ ]:
# Post-Processing Metrics: Bowling Average, Strike Rate, Economy

# Finding Bowling Average

T20_bowling_stats['bowling_average'] = (
    T20_bowling_stats['runs_conceded'] / T20_bowling_stats['wickets']
).replace([np.inf, np.nan], 0).round(2)


# Finding Bowler Strike rate

T20_bowling_stats['strike_rate'] = (
    T20_bowling_stats['balls_bowled'] / T20_bowling_stats['wickets']
).replace([np.inf, np.nan], 0).round(2)


# Finding Bowler Economy

T20_bowling_stats['economy'] = (
    T20_bowling_stats['runs_conceded'] / T20_bowling_stats['balls_bowled'] * 6
).replace([np.inf, np.nan], 0).round(2)


# Finding Wicket per balls

T20_bowling_stats['balls_per_wicket'] = (
    T20_bowling_stats['balls_bowled'] / T20_bowling_stats['wickets']
).replace([np.inf, np.nan], 0).round(2)


#Finding Boundaary Percentage

T20_bowling_stats['boundary_pct'] = (
    (T20_bowling_stats['fours_conceded'] + T20_bowling_stats['sixes_conceded']) /
    T20_bowling_stats['balls_bowled'] * 100
).replace([np.inf, np.nan], 0).round(2)

<h2><b><font color="gold">Bowler Vs Opponent Analysis</font></b></h2>

In [ ]:
# Bowler vs Opposition Analysis

T20_bowler_vs_opp = (
    df_T20_ball.groupby(['bowler', 'opposition']).agg(
        matches=('Match_ID', pd.Series.nunique),
        balls_bowled=('bowler', 'count'),
        runs_conceded=('total_runs', 'sum'),
        wickets=('is_wicket', 'sum'),
    ).reset_index()
)


In [ ]:
# Adding Bowling_Avg, Balls_Bowled, Economy fields to the dataframe

T20_bowler_vs_opp['bowling_average'] = (
    T20_bowler_vs_opp['runs_conceded'] / T20_bowler_vs_opp['wickets']
).replace([np.inf, np.nan], 0).round(2)
T20_bowler_vs_opp['strike_rate'] = (
    T20_bowler_vs_opp['balls_bowled'] / T20_bowler_vs_opp['wickets']
).replace([np.inf, np.nan], 0).round(2)
T20_bowler_vs_opp['economy'] = (
    T20_bowler_vs_opp['runs_conceded'] / T20_bowler_vs_opp['balls_bowled'] * 6
).replace([np.inf, np.nan], 0).round(2)

In [ ]:
# Wide Format (Pivot for Averages by Opposition)
T20_bowler_vs_opp_pivot = (
    T20_bowler_vs_opp.pivot(index='bowler', columns='opposition', values='bowling_average')
    .add_prefix('avg_vs_')
    .reset_index()
)
T20_bowling_summary = T20_bowling_stats.merge(T20_bowler_vs_opp_pivot, on='bowler', how='left')


<h2><b><font color="gold"> Bowler Vs venue Analysis</font></b></h2>

In [ ]:
# Bowler Vs venue

T20_bowler_vs_venue = (
    df_T20_ball.groupby(['bowler', 'venue']).agg(
        matches=('Match_ID', pd.Series.nunique),
        balls_bowled=('bowler', 'count'),
        runs_conceded=('total_runs', 'sum'),
        wickets=('is_wicket', 'sum'),
        fours_conceded=('batsman_runs', lambda x: (x == 4).sum()),
        sixes_conceded=('batsman_runs', lambda x: (x == 6).sum())
    ).reset_index()
)

In [ ]:
# Adding Bowling_Avg, Strike rate, Economy fields to the dataframe

T20_bowler_vs_venue['bowling_average'] = (
    T20_bowler_vs_venue['runs_conceded'] / T20_bowler_vs_venue['wickets']
).replace([np.inf, np.nan], 0).round(2)
T20_bowler_vs_venue['strike_rate'] = (
    T20_bowler_vs_venue['balls_bowled'] / T20_bowler_vs_venue['wickets']
).replace([np.inf, np.nan], 0).round(2)
T20_bowler_vs_venue['economy'] = (
    T20_bowler_vs_venue['runs_conceded'] / T20_bowler_vs_venue['balls_bowled'] * 6
).replace([np.inf, np.nan], 0).round(2)

In [ ]:
# Finding Recent Form from last 5 matches

N = 5
T20_innings_bowling['recent_form_wickets'] = (
    T20_innings_bowling.groupby('bowler')['wickets']
    .transform(lambda x: x.shift(1).rolling(window=N, min_periods=1).mean())
)
T20_innings_bowling['career_avg_wickets'] = (
    T20_innings_bowling.groupby('bowler')['wickets'].transform(lambda x: x.expanding().mean())
)